In [3]:
from tqdm import tqdm
from py2neo import Node, Relationship, Graph, Schema
import pandas as pd

class KnowledgeGraph(object):

    def __init__(self, url='bolt://localhost:7687', name='neo4j', auth=('neo4j','0000')):
        self.url = url
        self.name = name
        self.auth = auth
        self.graph = Graph(self.url, name=self.name, auth=self.auth)
        self.schema = Schema(self.graph)
        self.node_labels = ['O', 'VACC', 'SYMP', 'DIS']
        
        for label in self.node_labels:
            unique_constraints = self.schema.get_uniqueness_constraints(label)
            if "name" not in unique_constraints:
                self.schema.create_uniqueness_constraint(label, "name")

    def __call__(self, triples):
        """
        List[tuple]
        """
        for item in tqdm(triples, desc="GraphDB"):
            transaction = self.graph.begin()
            relation, subj_name, obj_name, subj_label, obj_label = item
            subj, obj = Node(subj_label, name=subj_name), Node(obj_label, name=obj_name)
            relationship = Relationship(subj, relation, obj)
            transaction.merge(subj, subj_label, "name")
            transaction.merge(obj, obj_label, "name")
            transaction.merge(relationship)
            self.graph.commit(transaction)

In [2]:
# 예시

# kg = KnowledgeGraph()
# triples=[('Triggers','화이자','근육통','VACC','SYMP'),('Triggers','화이자','몸살','VACC','SYMP')]
# kg(triples)

GraphDB: 100%|██████████| 2/2 [00:01<00:00,  1.34it/s]


In [4]:
kg_input_info = pd.read_csv('kg_input_required_info_filtered.tsv', sep='\t', encoding='utf-8')

re_tags = kg_input_info['re_tag']
e1s = kg_input_info['new_e1']
e2s = kg_input_info['new_e2']
e1_tags = kg_input_info['e1_tag']
e2_tags = kg_input_info['e2_tag']

triples = list(zip(re_tags, e1s, e2s, e1_tags, e2_tags))

kg = KnowledgeGraph()
kg(triples)

GraphDB: 100%|██████████| 859/859 [00:19<00:00, 44.17it/s]
